In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/national-park-trails/AllTrails data - nationalpark.csv')

In [ ]:
df.head()

Clean: look at the head of the file, we can see columns that are not numbers so we need to change:
    fill in nan data by 0
    1)loc: change to long and lat so that we can plot
    2)route_type: 3 unique values we can change to 2 bools 
    3)units: we can change to 1 bool column
    4)features: change to num_features and activities: change to num_activities
Exploratory
    a)scatterplot according to a map because we have longs and lats, hue=popularity
    b)see what elements affect popularity and plot those element against popularity
Model
    a)scale and split model
    b)create and train the model
    c)Evaluate
    d)Predict on new data
    

In [ ]:
df.describe()

In [ ]:
df.info()

Check for null values: the visitor_useage column has nan values so I will fill in 0. Please let me know if there are better options. Thank you!

In [ ]:
df.isnull().sum()

**Clean data**

In [ ]:
df['visitor_usage'] = df['visitor_usage'].fillna(0)

In [ ]:
#1
#for i in df['_geoloc']:
#    lat = float(i.split(',')[0].split()[-1])
df['lat'] = df['_geoloc'].apply(lambda i: float(i.split(',')[0].split()[-1]))

df['long'] = df['_geoloc'].apply(lambda i: float(i.split(',')[1].split()[-1][:-1]))

In [ ]:
df['route_type'].unique()

In [ ]:
#2
df['out_and_back'] = df['route_type'].apply(lambda i: 1 if i == 'out and back' else 0)
df['loop'] = df['route_type'].apply(lambda i: 1 if i == 'loop' else 0)

In [ ]:
df['units'].unique()

In [ ]:
#3
df['units'] = df['units'].apply(lambda i: 1 if i == 'i' else 0)

In [ ]:
#4 count the number of features
#for i in df['features']:
#    print(len(i.split(',')))
#both ways below are fine
#df['num_features']=df['features'].apply(lambda i: len(i.split(',')))
df['num_features']=df['features'].apply(lambda i: i.count(",") + 1)
#do the same for activities
df['num_activities']=df['activities'].apply(lambda i: (i.count(",") + 1))
    

**Explore the data**

we can see that there are different types of routes. Let's find out what kind of route is most popular.

In [ ]:
df['route_type'].nunique()

In [ ]:
dk = df.groupby('route_type')['popularity'].mean()

In [ ]:
dk.plot(kind='bar')

best states to trail based on average rating

In [ ]:
df.groupby('state_name')['avg_rating'].mean().sort_values(ascending=False)

best states to trail based on popularity

In [ ]:
df.groupby('state_name')['popularity'].mean().sort_values(ascending=False)

compare the popularity and the length of the trails. We can see that shorter trails are more popular than longer trails.

In [ ]:
sns.jointplot(x='length', y='popularity', data=df, alpha=0.5)

compare the elevation gain and popularity of trails. Most popular trails are under 2000 elevation gain

In [ ]:
sns.jointplot(x='elevation_gain', y='popularity', data=df, alpha=0.5)

popularity against number of reviews

In [ ]:
sns.jointplot(x='num_reviews', y='popularity', data=df, alpha=0.5)

In [ ]:
ratings = df[['name', 'avg_rating','num_reviews']].copy()

In [ ]:
ratings.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(ratings['num_reviews'])

most people gave a 4.5 rating.

In [ ]:
ratings['avg_rating'].hist(bins=20)

In [ ]:
sns.jointplot(x='avg_rating', y='num_reviews', data=ratings, alpha=0.5)

In [ ]:
ratings.sort_values('num_reviews',ascending=False).head(10)

In [ ]:
df1 = df.drop(['features','activities','_geoloc'],axis=1)

Discover the longs and lats

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(data=df, x='popularity', y='long')

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(data=df, x='popularity', y='lat') 

it was hard to see the distribution of popularity so we cut the data by top 1%, which is about 300 values

In [ ]:
df.sort_values('popularity',ascending=False).head()

In [ ]:
len(df)*0.1

In [ ]:
new_df = df.sort_values('popularity',ascending=False).iloc[300:]

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot(x='long',y='lat',
                data=new_df,hue='popularity',
                palette='RdYlGn',edgecolor=None,alpha=0.2)

In [ ]:
sns.boxplot(data=df, x='units', y='popularity')

In [ ]:
df1=df.drop(['_geoloc','features','activities','trail_id','name','area_name','city_name','state_name','country_name','route_type'],axis=1)

In [ ]:
df1['visitor_usage'] = df1['visitor_usage'].fillna(0)

In [ ]:
X=df1.drop('popularity',axis=1)
y=df1['popularity']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

In [ ]:
model=Sequential()
model.add(Dense(13,activation='relu'))
model.add(Dense(13,activation='relu'))
model.add(Dense(13,activation='relu'))
model.add(Dense(13,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(x=X_train,y=y_train.values,
         validation_data=(X_test,y_test.values),
         batch_size=128,epochs=400)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot()

**Evaluation**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [ ]:
X_test

In [ ]:
predictions=model.predict(X_test)

In [ ]:
mean_absolute_error(y_test,predictions)

In [ ]:
np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
explained_variance_score(y_test,predictions)

In [ ]:
df['popularity'].mean()

In [ ]:
df['popularity'].median()

In [ ]:
plt.scatter(y_test,predictions)
plt.plot(y_test,y_test,'r')

In [ ]:
errors=y_test.values.reshape(994,1) - predictions
sns.distplot(errors)

If there is a new trail road, predict the popularity on the page of that new trail

In [ ]:
new_trail=df1.drop('popularity',axis=1).iloc[0]
new_trail=scaler.transform(new_trail.values.reshape(-1,13))
new_trail

In [ ]:
model.predict(new_trail)

In [ ]:
df1.iloc[0]

Thank you for reading! Please let me know what I can do to make it better!